# `SENTIMENTAL ANALYSIS`

#### `IMPORTIN LIBRARIES`

In [1]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from textblob import Word
import pickle
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import SGD
import keras.backend as K
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### `IMPORTING DATASET`

In [2]:
ds=pd.read_csv("tweet_train.csv",encoding="ISO-8859-1",names=["label", "ids", "date", "flag", "user", "tweet"])
ds=ds.drop(["ids","date","flag","user"],axis=1)

In [3]:
positive=ds[ds["label"]==0][:1000]
neagtive=ds[ds["label"]==4][:1000]

In [4]:
positive.info()
print()
neagtive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1000 non-null   int64 
 1   tweet   1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 23.4+ KB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 800000 to 800999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1000 non-null   int64 
 1   tweet   1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 23.4+ KB


In [5]:
positive

,label,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
995,0,@dkoenigs thanks man. I'm so very grateful. ...
996,0,@t_wolfe i miss u too. i'm totally comin back...
997,0,@sniffinglue ohhh. I love it. ps I'm sad we di...
998,0,And somehow I still end up in this place


In [6]:
neagtive["label"]=neagtive["label"].apply(lambda x:1)
neagtive

,label,tweet
800000,1,I LOVE @Health4UandPets u guys r the best!!
800001,1,im meeting up with one of my besties tonight! ...
800002,1,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,1,Being sick can be really cheap when it hurts t...
800004,1,@LovesBrooklyn2 he has that effect on everyone
...,...,...
800995,1,I have this strange desire to go to confession...
800996,1,@i_reporter answer sent in dm. try it
800997,1,@brooklynunion cuz ur 3pm is my 9am and Id be ...
800998,1,@littrellfans Its all good. Just figured you w...


In [7]:
dataset=pd.concat([positive,neagtive])
dataset=dataset.sample(frac=1)
dataset

,label,tweet
800010,1,crazy day of school. there for 10 hours straii...
156,0,"getting sick time for some hot tea, studying,..."
326,0,I wish I was better at writing. It's taking me...
427,0,@maxime68 @megelder But! I'm useless if I don'...
800760,1,@deepbluesealove I was never an early morning ...
...,...,...
800604,1,@EmilyCello Nessun realmente! ... I guess &quo...
800903,1,@RepeaterBand I missed the show last Friday! I...
265,0,@chriscantore congrats! I'm totally jealous! o...
551,0,@ginayates Sorry to hear about Maggie. Thou...


### `PRE-PROCESSING`

##### `PRE PROCESSING VARIABLE SETTING`

In [8]:
# dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+",' ',x))

In [9]:
dataset

,label,tweet
800010,1,crazy day of school there for 10 hours straiii...
156,0,getting sick time for some hot tea studying an...
326,0,I wish I was better at writing It s taking me ...
427,0,megelder But I m useless if I don t sleep It...
800760,1,I was never an early morning person so I rea...
...,...,...
800604,1,Nessun realmente I guess quot No Davvvero qu...
800903,1,I missed the show last Friday I was so bumme...
265,0,congrats I m totally jealous only wish my XM...
551,0,Sorry to hear about Maggie Thoughts to your ...


In [10]:
dataset["tweet"]=dataset["tweet"].apply(lambda x: " ".join([w for w in x.split() if w not in stopwords.words("english")]))
dataset

,label,tweet
800010,1,crazy day school 10 hours straiiight watch hil...
156,0,getting sick time hot tea studying sleeeep
326,0,I wish I better writing It taking long write p...
427,0,megelder But I useless I sleep It fair I want ...
800760,1,I never early morning person I really didnt an...
...,...,...
800604,1,Nessun realmente I guess quot No Davvvero quot...
800903,1,I missed show last Friday I bummed But I excit...
265,0,congrats I totally jealous wish XM working
551,0,Sorry hear Maggie Thoughts mum


In [11]:
# Lemmatizing

dataset['tweet']=dataset['tweet'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
dataset

,label,tweet
800010,1,crazy day school 10 hour straiiight watch hill...
156,0,getting sick time hot tea studying sleeeep
326,0,I wish I better writing It taking long write p...
427,0,megelder But I useless I sleep It fair I want ...
800760,1,I never early morning person I really didnt an...
...,...,...
800604,1,Nessun realmente I guess quot No Davvvero quot...
800903,1,I missed show last Friday I bummed But I excit...
265,0,congrats I totally jealous wish XM working
551,0,Sorry hear Maggie Thoughts mum


### `SPLITTING DATA FOR TRAINING AND TESTING`

#### `LSTM MODEL`

In [12]:
max_words = 2000
max_len=50

def tokenize_pad_sequences(text):
    tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
    tokenizer.fit_on_texts(text)
    X = tokenizer.texts_to_sequences(text)
    X = pad_sequences(X, padding='post', maxlen=max_len)
    return X, tokenizer

print('Before Tokenization & Padding \n', dataset['tweet'][0])
X, tokenizer = tokenize_pad_sequences(dataset['tweet'])
print('After Tokenization & Padding \n', X[0])

Before Tokenization & Padding 
 Awww bummer You shoulda got David Carr Third Day D
After Tokenization & Padding 
 [ 168    3   65  210   60 1724  169  330 1725  331  495   68  373 1726
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [13]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [14]:
vocab_size = 2000
embedding_size = 32
epochs=20
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model= Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_len))
model.add(Conv1D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

c:\Users\ganes\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [15]:
from sklearn.model_selection import train_test_split
Y=pd.get_dummies(dataset["label"]).values
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(1600, 50) (1600, 2)
(400, 50) (400, 2)


In [16]:
model.compile(loss='binary_crossentropy', optimizer=sgd, 
               metrics=['accuracy', Precision(), Recall()])


In [17]:
model.fit(X_train, y_train,
                      validation_data=(X_test, y_test),
                      batch_size=10, epochs=30, verbose=1)

Epoch 1/30


: 

: 

In [ ]:
twt = ['he is the such a bad guy']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
# model.save("lstm_model.h5")